# ChatGPT Like app Via Streamlit
Project Structure would be -

1. app.py
2. requirements.txt
3. .streamlit > secrets.toml
4. chat-history (folder)


app.py file below (discussed in the class)

In [ ]:
# Multi-chat app with persistent history using OpenRouter
# Features: Multiple conversations, persistent storage, chat history in sidebar

import streamlit as st
from openai import OpenAI
import os
import json
from datetime import datetime
from pathlib import Path

# Configure the page
st.set_page_config(page_title="My ChatBot", page_icon="🤖", layout="wide")

# Initialize the OpenAI client with OpenRouter
try:
    api_key = st.secrets["OPENROUTER_API_KEY"]
except Exception:
    st.error("OPENROUTER_API_KEY not found in .streamlit/secrets.toml")
    st.stop()

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
    default_headers={
        "HTTP-Referer": "http://localhost:8504",
        "X-Title": "My ChatBot",
    }
)

# Setup persistent storage directory
CHAT_STORAGE_DIR = Path(__file__).parent / "chat_history"
CHAT_STORAGE_DIR.mkdir(exist_ok=True)

# ============================================================================
# CHAT PERSISTENCE FUNCTIONS
# ============================================================================

def get_all_chats():
    """Get all chat files sorted by modification time (newest first)"""
    chat_files = list(CHAT_STORAGE_DIR.glob("chat_*.json"))
    chat_files.sort(key=lambda x: x.stat().st_mtime, reverse=True)
    return chat_files

def load_chat(chat_id):
    """Load a specific chat by ID"""
    chat_file = CHAT_STORAGE_DIR / f"chat_{chat_id}.json"
    if chat_file.exists():
        with open(chat_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            return data
    return None

def save_chat(chat_id, messages, title=None):
    """Save chat to disk"""
    chat_file = CHAT_STORAGE_DIR / f"chat_{chat_id}.json"

    # Auto-generate title from first user message if not provided
    if title is None and messages:
        for msg in messages:
            if msg["role"] == "user":
                title = msg["content"][:50] + ("..." if len(msg["content"]) > 50 else "")
                break

    if title is None:
        title = "New Chat"

    data = {
        "chat_id": chat_id,
        "title": title,
        "messages": messages,
        "created_at": datetime.now().isoformat(),
        "updated_at": datetime.now().isoformat()
    }

    # If file exists, preserve created_at
    if chat_file.exists():
        with open(chat_file, 'r', encoding='utf-8') as f:
            old_data = json.load(f)
            data["created_at"] = old_data.get("created_at", data["created_at"])

    with open(chat_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def delete_chat(chat_id):
    """Delete a chat file"""
    chat_file = CHAT_STORAGE_DIR / f"chat_{chat_id}.json"
    if chat_file.exists():
        chat_file.unlink()

def create_new_chat():
    """Create a new chat with unique ID"""
    chat_id = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
    return chat_id

def get_chat_title(chat_data):
    """Extract chat title from chat data"""
    return chat_data.get("title", "Untitled Chat")

# ============================================================================
# SESSION STATE INITIALIZATION
# ============================================================================

# Initialize current chat ID
if "current_chat_id" not in st.session_state:
    # Try to load the most recent chat, or create new one
    all_chats = get_all_chats()
    if all_chats:
        latest_chat = load_chat(all_chats[0].stem.replace("chat_", ""))
        st.session_state.current_chat_id = latest_chat["chat_id"]
        st.session_state.messages = latest_chat["messages"]
        st.session_state.chat_title = latest_chat["title"]
    else:
        st.session_state.current_chat_id = create_new_chat()
        st.session_state.messages = []
        st.session_state.chat_title = "New Chat"

# Initialize messages
if "messages" not in st.session_state:
    st.session_state.messages = []

# Initialize chat title
if "chat_title" not in st.session_state:
    st.session_state.chat_title = "New Chat"

# Initialize feedback
if "feedback" not in st.session_state:
    st.session_state.feedback = {}

# Initialize dark mode
if "dark_mode" not in st.session_state:
    st.session_state.dark_mode = True

# ============================================================================
# SIDEBAR: CHAT MANAGEMENT
# ============================================================================

with st.sidebar:
    st.header("💬 Conversations")

    # New Chat button
    if st.button("➕ New Chat", use_container_width=True, type="primary"):
        # Save current chat before creating new one
        if st.session_state.messages:
            save_chat(
                st.session_state.current_chat_id,
                st.session_state.messages,
                st.session_state.chat_title
            )

        # Create new chat
        st.session_state.current_chat_id = create_new_chat()
        st.session_state.messages = []
        st.session_state.chat_title = "New Chat"
        st.session_state.feedback = {}
        st.rerun()

    st.divider()

    # List all chats
    st.subheader("Chat History")
    all_chats = get_all_chats()

    if all_chats:
        for chat_file in all_chats:
            chat_id = chat_file.stem.replace("chat_", "")
            chat_data = load_chat(chat_id)

            if chat_data:
                chat_title = get_chat_title(chat_data)
                is_current = chat_id == st.session_state.current_chat_id

                col1, col2 = st.columns([4, 1])

                with col1:
                    # Show current chat with indicator
                    button_label = f"{'🟢 ' if is_current else ''}{chat_title}"
                    if st.button(
                        button_label,
                        key=f"load_{chat_id}",
                        use_container_width=True,
                        disabled=is_current,
                        type="secondary" if is_current else "tertiary"
                    ):
                        # Save current chat before switching
                        if st.session_state.messages:
                            save_chat(
                                st.session_state.current_chat_id,
                                st.session_state.messages,
                                st.session_state.chat_title
                            )

                        # Load selected chat
                        st.session_state.current_chat_id = chat_id
                        st.session_state.messages = chat_data["messages"]
                        st.session_state.chat_title = chat_title
                        st.session_state.feedback = {}
                        st.rerun()

                with col2:
                    # Delete button (only for non-current chats or if it's the only chat)
                    if st.button("🗑️", key=f"delete_{chat_id}", help="Delete chat"):
                        delete_chat(chat_id)

                        # If we deleted the current chat, switch to another or create new
                        if chat_id == st.session_state.current_chat_id:
                            remaining_chats = [c for c in all_chats if c.stem.replace("chat_", "") != chat_id]
                            if remaining_chats:
                                new_chat_data = load_chat(remaining_chats[0].stem.replace("chat_", ""))
                                st.session_state.current_chat_id = new_chat_data["chat_id"]
                                st.session_state.messages = new_chat_data["messages"]
                                st.session_state.chat_title = new_chat_data["title"]
                            else:
                                st.session_state.current_chat_id = create_new_chat()
                                st.session_state.messages = []
                                st.session_state.chat_title = "New Chat"
                            st.session_state.feedback = {}

                        st.rerun()
    else:
        st.info("No chat history yet. Start a new conversation!")

    st.divider()

    # Settings section
    st.subheader("⚙️ Settings")
    dark_mode = st.toggle("Dark mode", value=st.session_state.dark_mode)
    st.session_state.dark_mode = dark_mode

    # Clear current chat
    if st.button("🗑️ Clear Current Chat", use_container_width=True):
        st.session_state.messages = []
        st.session_state.feedback = {}
        st.session_state.chat_title = "New Chat"
        save_chat(st.session_state.current_chat_id, [], "New Chat")
        st.rerun()

# ============================================================================
# APPLY THEMING
# ============================================================================

if st.session_state.dark_mode:
    st.markdown(
        """
        <style>
        .stApp { background-color: #0f1115; color: #e6e6e6; }
        .stChatMessage, .stMarkdown { color: #e6e6e6; }
        </style>
        """,
        unsafe_allow_html=True,
    )

# ============================================================================
# MAIN CHAT INTERFACE
# ============================================================================

# App title with current chat title
st.title(f"🤖 {st.session_state.chat_title}")

# Summarize conversation - for entire current chat
with st.expander("📝 Summarize Conversation", expanded=False):
    st.write("Generate a summary of the entire conversation in this chat")

    if st.button("Generate Summary", use_container_width=True):
        if not st.session_state.messages:
            st.warning("No messages to summarize yet!")
        else:
            try:
                with st.spinner("Generating summary..."):
                    summary_resp = client.chat.completions.create(
                        model="openai/gpt-oss-120b",
                        messages=[
                            {"role": "system", "content": "Summarize the conversation into concise key points and action items."},
                            *st.session_state.messages,
                        ],
                        stream=False,
                        extra_body={}
                    )
                    summary_text = summary_resp.choices[0].message.content.strip()
                    st.markdown("### Summary")
                    st.markdown(summary_text)
            except Exception as e:
                st.error(f"Summary failed: {e}")

# Display chat history
for idx, message in enumerate(st.session_state.messages):
    with st.chat_message(message["role"]):
        st.markdown(message["content"])
        if message["role"] == "assistant":
            c1, c2 = st.columns([1, 1])
            with c1:
                if st.button("👍", key=f"up_{idx}"):
                    st.session_state.feedback[idx] = "up"
            with c2:
                if st.button("👎", key=f"down_{idx}"):
                    st.session_state.feedback[idx] = "down"

# Handle user input
if prompt := st.chat_input("What would you like to know?"):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    # Update chat title if this is the first message
    if len(st.session_state.messages) == 1:
        st.session_state.chat_title = prompt[:50] + ("..." if len(prompt) > 50 else "")

    # Display user message
    with st.chat_message("user"):
        st.markdown(prompt)

    # Generate AI response
    with st.chat_message("assistant"):
        try:
            response = client.chat.completions.create(
                model="openai/gpt-oss-120b",
                messages=st.session_state.messages,
                stream=True,
                extra_headers={
                    "HTTP-Referer": "http://localhost:8503",
                    "X-Title": "My ChatBot"
                },
                extra_body={}
            )

            # Stream the response
            response_text = ""
            response_placeholder = st.empty()

            for chunk in response:
                if chunk.choices[0].delta.content is not None:
                    # Clean up unwanted tokens
                    content = chunk.choices[0].delta.content
                    content = (
                        content.replace('<s>', '')
                        .replace('<|im_start|>', '')
                        .replace('<|im_end|>', '')
                        .replace("<|OUT|>", "")
                    )
                    response_text += content
                    response_placeholder.markdown(response_text + "▌")

            # Final cleanup of response text
            response_text = (
                response_text.replace('<s>', '')
                .replace('<|im_start|>', '')
                .replace('<|im_end|>', '')
                .replace("<|OUT|>", "")
                .strip()
            )
            response_placeholder.markdown(response_text)

            # Add assistant response to chat history
            st.session_state.messages.append(
                {"role": "assistant", "content": response_text}
            )

            # Save chat to disk
            save_chat(
                st.session_state.current_chat_id,
                st.session_state.messages,
                st.session_state.chat_title
            )

        except Exception as e:
            st.error(f"Error: {str(e)}")
            st.info("Please check your API key and try again.")

# Auto-save chat when messages change (backup mechanism)
if st.session_state.messages:
    save_chat(
        st.session_state.current_chat_id,
        st.session_state.messages,
        st.session_state.chat_title
    )


# Challenge 1

# Challenge 1: Translation Mode

**Difficulty**: Intermediate
**Estimated Time**: 30-40 minutes
**Focus**: System prompts, multiple API calls, language detection

## Challenge Description

Transform your basic chatbot into an intelligent translation assistant that can automatically detect languages and provide high-quality translations with cultural context.

## User Story

*"As a user, I want to type text in any language and have the chatbot automatically detect the language and offer to translate it to my preferred target language. The bot should also provide cultural context and alternative translations when relevant."*

## Requirements

### Core Features (Must Have)
- [x] **Language Detection**: Automatically identify the input language
- [x] **Translation**: Translate text to user-selected target language
- [x] **Language Selection**: Sidebar control for target language selection
- [x] **Bidirectional Translation**: Support translation in both directions

### Advanced Features (Nice to Have)
- [x] **Cultural Context**: Provide cultural notes for idiomatic expressions
- [x] **Alternative Translations**: Offer multiple translation options
- [x] **Confidence Scoring**: Show confidence level of detection/translation
- [x] **Translation History**: Keep track of translation pairs

## Technical Approach

You'll need to modify your existing chatbot to:

1. **System Prompt Engineering**: Create a specialized prompt for translation tasks
2. **Two-Stage Process**: First detect language, then translate
3. **State Management**: Track source/target languages in session state
4. **UI Enhancements**: Add language selection controls

## Key Learning Objectives

- Master system prompt engineering for specialized tasks
- Handle multiple API calls in sequence
- Implement conditional logic based on AI responses
- Create professional translation UX patterns

## Getting Started

1. Copy your working chatbot from the main workshop
2. Add language selection to the sidebar
3. Modify the system prompt for translation tasks
4. Implement the two-stage detection + translation process

## Example Interactions

**Input**: "Bonjour, comment allez-vous?"
**Output**:
```
🔍 Detected Language: French
🎯 Translation (English): "Hello, how are you?"

💡 Cultural Note: This is a formal greeting in French. In casual settings,
   you might hear "Salut, ça va?" instead.
```

**Input**: "I love this weather"
**Output**:
```
🔍 Detected Language: English
🎯 Translation (Spanish): "Me encanta este clima"

🌟 Alternative: "Adoro este tiempo" (more emphatic)
💡 Regional Note: In Mexico, you might also hear "está padrísimo el clima"
```

## Hints Available

- Progressive hints in `hints.md`
- Complete solution in `solution.py`
- Don't peek at the solution until you've tried for at least 20 minutes!

## Success Criteria

Your translation bot should:
✅ Automatically detect input language
✅ Translate accurately to target language
✅ Provide cultural context when relevant
✅ Handle errors gracefully
✅ Maintain conversation history
✅ Have intuitive language selection UI

## Extension Ideas

- Add support for document translation
- Implement translation confidence scoring
- Create translation glossaries for technical terms
- Add pronunciation guides
- Support batch translation of multiple texts

Good luck! Remember, the goal is to learn the patterns - don't be afraid to experiment and iterate.

# Challenge 2

# Challenge 2: Personality Selector

**Difficulty**: Beginner-Intermediate
**Estimated Time**: 25-35 minutes
**Focus**: Session state management, system prompts, UI controls

## Challenge Description

Transform your chatbot into a versatile assistant with multiple personalities that users can select from. Each personality should have a distinct communication style, expertise area, and approach to helping users.

## User Story

*"As a user, I want to choose from different AI personality modes (like Professional, Creative, Technical, etc.) so that I get responses that match the style and expertise I need for my current task."*

## Requirements

### Core Features (Must Have)
- [x] **Personality Selection**: Sidebar dropdown with at least 4 personality options
- [x] **Dynamic System Prompts**: Each personality uses a different system prompt
- [x] **Visual Indicators**: Show current personality in the chat interface
- [x] **Personality Persistence**: Remember selected personality across conversation

### Advanced Features (Nice to Have)
- [x] **Custom Personality**: Allow users to define their own personality
- [x] **Personality Descriptions**: Show what each personality is good for
- [x] **Response Style Preview**: Show example responses for each personality
- [x] **Personality Switching**: Allow mid-conversation personality changes

## Personality Options

### 1. Professional Business Assistant
- **Style**: Formal, structured, business-focused
- **Expertise**: Business strategy, professional communication
- **Tone**: Polite, efficient, results-oriented

### 2. Creative Writing Helper
- **Style**: Imaginative, expressive, inspiring
- **Expertise**: Creative writing, storytelling, artistic projects
- **Tone**: Enthusiastic, artistic, encouraging

### 3. Technical Expert
- **Style**: Precise, detailed, code-focused
- **Expertise**: Programming, technology, problem-solving
- **Tone**: Analytical, methodical, educational

### 4. Friendly Companion
- **Style**: Casual, supportive, conversational
- **Expertise**: General chat, emotional support, casual advice
- **Tone**: Warm, empathetic, encouraging

### 5. Custom Personality (Advanced)
- **Style**: User-defined
- **Expertise**: User-specified
- **Tone**: Customizable

## Technical Approach

You'll modify your existing chatbot to:

1. **Add Personality State**: Track selected personality in session state
2. **System Prompt Engineering**: Create distinct prompts for each personality
3. **UI Enhancement**: Add sidebar controls for personality selection
4. **Message Context**: Inject personality context into conversation

## Key Learning Objectives

- Master system prompt engineering for different use cases
- Implement dynamic behavior based on user selections
- Create intuitive personality switching UX
- Understand how context shapes AI responses

## Getting Started

1. Copy your working chatbot from the main workshop
2. Add personality selection to the sidebar
3. Create system prompts for each personality
4. Implement dynamic prompt switching

## Example Interactions

**Professional Mode**:
```
User: "How do I handle a difficult client meeting?"
Assistant: "I recommend a structured approach to managing challenging client interactions:

1. Preparation: Review all relevant documentation beforehand
2. Active Listening: Allow the client to express their concerns fully
3. Solution-Focused Discussion: Present 2-3 concrete options
4. Clear Next Steps: Document agreed actions with timelines

Would you like me to elaborate on any of these strategies?"
```

**Creative Mode**:
```
User: "How do I handle a difficult client meeting?"
Assistant: "Ah, the art of navigating stormy client waters! 🎭

Picture this: You're not just attending a meeting, you're crafting a story where everyone wins. Start by becoming a master listener - let their frustrations paint the full picture. Then, like a skilled storyteller, weave their concerns into solutions that feel like plot twists they never saw coming!

Transform the tension into creative energy. What if this 'difficult' meeting becomes the turning point where you surprise them with innovative approaches they hadn't considered? ✨"
```

## Success Criteria

Your personality selector should:
✅ Offer at least 4 distinct personalities
✅ Show clear differences in response style
✅ Maintain personality consistency throughout conversation
✅ Allow personality switching mid-conversation
✅ Display current personality clearly to users
✅ Handle personality changes gracefully

## Extension Ideas

- Add personality-specific conversation starters
- Implement personality learning (adapt based on user feedback)
- Create personality profiles with detailed descriptions
- Add visual avatars for each personality
- Enable personality mixing (combine traits from multiple personalities)

Remember: The goal is to understand how system prompts shape AI behavior. Experiment with different prompt styles and see how dramatically they change the responses!

# Challenge 3

# Challenge 3: Export Functionality

**Difficulty**: Intermediate-Advanced
**Estimated Time**: 35-45 minutes
**Focus**: Data formatting, file operations, download handling

## Challenge Description

Add comprehensive export functionality to your chatbot that allows users to download their conversation history in multiple formats with rich metadata and formatting options.

## User Story

*"As a user, I want to export my conversation history in different formats (TXT, JSON, CSV) so I can save important discussions, share them with others, or import them into other tools for analysis."*

## Requirements

### Core Features (Must Have)
- [x] **Multiple Export Formats**: TXT, JSON, and CSV export options
- [x] **Download Functionality**: Use `st.download_button` for direct downloads
- [x] **Formatted Output**: Clean, readable formatting for each format
- [x] **Metadata Inclusion**: Timestamps, message counts, session info

### Advanced Features (Nice to Have)
- [x] **Export Filtering**: Allow users to export specific date ranges or message types
- [x] **Rich Text Formatting**: Markdown preservation, code blocks, special formatting
- [x] **Statistics Summary**: Include conversation analytics in exports
- [x] **Batch Export**: Export multiple conversations or sessions

## Export Format Specifications

### 1. TXT Format (Human Readable)
```
Chat Export - 2024-01-15 14:30
========================================

Session Information:
- Total Messages: 12
- Duration: 25 minutes
- Export Date: 2024-01-15 14:55

Conversation:
----------------------------------------

[14:30:15] You: Hello! How can I help you today?

[14:30:22] Assistant: Hello! I'm here to help you with any questions or tasks you have. What would you like to discuss?

[14:30:45] You: Can you explain machine learning?

[14:31:02] Assistant: Machine learning is a subset of artificial intelligence...
```

### 2. JSON Format (Structured Data)
```json
{
  "export_metadata": {
    "export_timestamp": "2024-01-15T14:55:30Z",
    "format_version": "1.0",
    "session_id": "session_123",
    "total_messages": 12,
    "session_duration_minutes": 25
  },
  "conversation": [
    {
      "timestamp": "2024-01-15T14:30:15Z",
      "role": "user",
      "content": "Hello! How can I help you today?",
      "message_id": 1,
      "character_count": 32
    },
    {
      "timestamp": "2024-01-15T14:30:22Z",
      "role": "assistant",
      "content": "Hello! I'm here to help you...",
      "message_id": 2,
      "character_count": 95
    }
  ],
  "statistics": {
    "user_messages": 6,
    "assistant_messages": 6,
    "total_characters": 2847,
    "average_message_length": 237
  }
}
```

### 3. CSV Format (Data Analysis)
```csv
Message_ID,Timestamp,Role,Content,Character_Count,Word_Count
1,2024-01-15 14:30:15,user,"Hello! How can I help you today?",32,8
2,2024-01-15 14:30:22,assistant,"Hello! I'm here to help you...",95,18
```

## Technical Approach

You'll need to implement:

1. **Data Processing**: Convert session state messages to exportable formats
2. **File Generation**: Create properly formatted files in memory
3. **Download Interface**: Use Streamlit's download functionality
4. **Export Options**: Allow users to customize what gets exported

## Key Learning Objectives

- Master data transformation and formatting
- Implement file generation and download patterns
- Create user-friendly export interfaces
- Handle different data formats and their use cases

## Getting Started

1. Copy your working chatbot from the main workshop
2. Add export controls to the sidebar
3. Implement basic TXT export first
4. Add JSON and CSV formats progressively

## Example Implementation Structure

```python
def export_as_txt(messages, metadata):
    """Convert messages to formatted text"""
    # Implementation here
    return text_content

def export_as_json(messages, metadata):
    """Convert messages to structured JSON"""
    # Implementation here
    return json_content

def export_as_csv(messages, metadata):
    """Convert messages to CSV format"""
    # Implementation here
    return csv_content

# In sidebar
if st.button("📤 Export Chat"):
    if format_choice == "TXT":
        content = export_as_txt(st.session_state.messages, metadata)
        st.download_button("💾 Download TXT", content, "chat.txt")
```

## Success Criteria

Your export system should:
✅ Generate clean, readable exports in all three formats
✅ Include relevant metadata and timestamps
✅ Handle edge cases (empty conversations, special characters)
✅ Provide intuitive export options and filters
✅ Generate properly formatted files for each format
✅ Calculate and include conversation statistics

## Extension Ideas

- **Email Integration**: Send exports via email
- **Cloud Storage**: Upload exports to Google Drive, Dropbox
- **Template System**: Custom export templates
- **Compression**: ZIP multiple formats together
- **Scheduling**: Automatic periodic exports
- **Import Functionality**: Import previous conversations

## Format-Specific Considerations

### TXT Format
- Human-readable timestamps
- Clear conversation flow
- Preserve formatting and line breaks
- Include conversation statistics

### JSON Format
- Valid JSON structure
- Rich metadata
- Extensible for future features
- Machine-readable timestamps

### CSV Format
- Proper escaping of commas and quotes
- Headers for data analysis tools
- Consistent data types
- Easy import into spreadsheets

Remember: This challenge is about data handling and user experience. Focus on creating exports that are genuinely useful for different purposes!